In [1]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from IPython.display import display,Math, Latex
import numpy as np

### PCA

* Tecnica para reducir las dimensiones de un conjunto de datos
* se encarga de combinar los datos y buscar la proyeccion segun la cual los datos queden mejor representados en terminos de minimos cuadraticos. buscando un conjunto de variables sin correlacion lineal, que con un minimo numero de variables explique lo maximo del dataset.
* Tecnica utilizada para descubir un conjunto de datos en terminos de nuevas variables, de nuevas componentes que no esten correlacionadas entre si
* cuanto mayor sea la variabilidad de los datos o varianza, se considera que hay mayor informacio, esto relacionado al concepto de entropia.

#### La solucion de Pearson
para estudair las relaciones que se presentan entre p variables correlacionadas, se puede transfrmar el conjunto original de variables en otro conjunto de nuevas variables incorreladas entre si (que no tengan redundancia en la informacion), llamado conjunto de componentes principales.

* las nuevas variables son combinaciones lineales de las anteriores y se van construyendo segun el orden de importancia en cunato a la variablidad total que recogen de la muestra.

* no requiere la condicion de normalidad de las variables ni normalidad multivariante, aunque si esto se cumple se puede dar una interpretacion mas precisa de las componentes.

nuevas variables:

In [15]:
display(Math(r'y_{i} = \sum_{j=1}^{m}a_{ij}x_{j} = a_{i}x'))

<IPython.core.display.Math object>

* se busca que las nuevas variabales tengan correlacion nula y que las varianzas vayan decreciendo paulatinamente.
* se impone una restriccion ya que lo que quiero es maximizar la varianza,y para evitar que aumentar la varianza se traduzca en aumentar demasiado los ai, es mantener la ortogonalidad de la transformacion imponiendo que le modulo de ai sea 1 (la suma de los cuadrados de cada vector ai)
* se resuleve el problema de optimizacion planteado con la tecnica de los multiplicadores de lagrange, para resolver el problema de maximizacion de las varianzas respectivas de cada nueva variable creada.
* El método de los multiplicadores de Lagrange o método de Lagrange nos permite encontrar los máximos y mínimos de una función multivariable cuando hay alguna restricción en los valores de entrada que puede usar.

### PCA FROM SCRATCH

El objetivo es identificar patrones y correlaciones entre variables, de modo que variables que esten muy correlacionadas las intentamos juntar en una nueva para ir quitando dimensiones.
Encontrar las direcciones de maxima variabilidad en espacios vectoriales de muchas dimensiones y proyectar toda esa elevada dimensionalidad en subespacios vectoriales de dimension inferior intendando retener siempre la mayor cantidad de información.
* Analisis del discriminador lineal LDA: mientras que PCA intenta identificar las componentes principales que maximizan la varianza del dato, el LDA intenta buscar direcciones que maximicen la separacion, la discriminacion entre las diferentes clases. Es muy util cuando se deben aplicar algoritmos de clasificacion de patrones.

**Pasos:**
1. Estandarizar los datos
2. Obtener los vectores y valores propios a partir de la matriz de covarianzas o de correlaciones, o la tecnica singular vector decomposition.
3. Ordenar los valores propios de manera descendente y quedarnos con los p que se correspondan a los p mayores y asi disminuir el numero de variables del dataset (p<m)
4. Construir la matriz de proyeccion W a partir de los p vectores propios.
5. Transformar el dataset original X a traves de W para asi obtener datos en el subespacio dimensional de dimension p, que será Y

In [2]:
df = pd.read_csv("../datasets/iris/iris.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  150 non-null    float64
 1   Sepal.Width   150 non-null    float64
 2   Petal.Length  150 non-null    float64
 3   Petal.Width   150 non-null    float64
 4   Species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [64]:
X = df.iloc[:,0:4].values
y = df.iloc[:,4].values

In [65]:
X[0]

array([5.1, 3.5, 1.4, 0.2])

In [66]:
fig = px.histogram(df, x='Sepal.Length', color='Species')
fig.show()

In [7]:
# estandarizar
X_std= StandardScaler().fit_transform(X)

#### 1. Descomposicion de valores y vectores propios
##### a) Usando mariz de covarianzas
A partir de la matriz de covarianza se extraen los vectores propios que determinan las direcciones en el nuevo espacio vectorial. Mientras que los valores propios determinan la magnitud, explican la varianza de los datos a traves de los nuevos ejes.


In [19]:
display(Math(r'\sigma_{jk} = \frac{1}{n-1}\sum_{i=1}^m (x_{ij}-\overline{x_j})(x_{ik}-\overline{x_k})'))

<IPython.core.display.Math object>

In [21]:
display(Math(r'\sigma_{jk} = \frac{1}{n-1}(X-\overline{x})^T(X-\overline{x})'))

<IPython.core.display.Math object>

In [25]:
display(Math(r'\overline{x} = \sum_{i=1}^n x_i\in \mathbb R^m'))

<IPython.core.display.Math object>

In [13]:
mean_vect = np.mean(X_std,axis=0)

In [67]:
cov_matrix = ((X_std-mean_vect).T.dot(X_std-mean_vect))/(X_std.shape[0]-1)

In [68]:
cov_matrix

array([[ 1.00671141, -0.11835884,  0.87760447,  0.82343066],
       [-0.11835884,  1.00671141, -0.43131554, -0.36858315],
       [ 0.87760447, -0.43131554,  1.00671141,  0.96932762],
       [ 0.82343066, -0.36858315,  0.96932762,  1.00671141]])

In [69]:
eig_vals, eig_vectors = np.linalg.eig(cov_matrix)
print("valores propios \n%s" %eig_vals)
print("vectores propios \n%s" %eig_vectors)

valores propios 
[2.93808505 0.9201649  0.14774182 0.02085386]
vectores propios 
[[ 0.52106591 -0.37741762 -0.71956635  0.26128628]
 [-0.26934744 -0.92329566  0.24438178 -0.12350962]
 [ 0.5804131  -0.02449161  0.14212637 -0.80144925]
 [ 0.56485654 -0.06694199  0.63427274  0.52359713]]


#### b) Usando Matriz de Correlaciones
La matriz de correlaciones es una version normalizada de la matriz de covarianza

In [59]:
corr_matrix = np.corrcoef(X_std.T)
corr_matrix

array([[ 1.        , -0.11756978,  0.87175378,  0.81794113],
       [-0.11756978,  1.        , -0.4284401 , -0.36612593],
       [ 0.87175378, -0.4284401 ,  1.        ,  0.96286543],
       [ 0.81794113, -0.36612593,  0.96286543,  1.        ]])

In [61]:
eig_vals_corr, eig_vectors_corr = np.linalg.eig(corr_matrix)
print("valores propios \n%s" %eig_vals_corr)
print("vectores propios \n%s" %eig_vectors_corr)

valores propios 
[2.91849782 0.91403047 0.14675688 0.02071484]
vectores propios 
[[ 0.52106591 -0.37741762 -0.71956635  0.26128628]
 [-0.26934744 -0.92329566  0.24438178 -0.12350962]
 [ 0.5804131  -0.02449161  0.14212637 -0.80144925]
 [ 0.56485654 -0.06694199  0.63427274  0.52359713]]


#### b) Singular Value decomposition

In [8]:
u,s,v = np.linalg.svd(X_std.T)

### 2. Las componentes principales 

El objetivo es reducir la dimensionalidad del espeacio vectorial original, proyectando los datos en espacios de dimension mas pequeños desde las direcciones que definen los vectores propios. 
Los vectores propios no solo definen la direccion sino que ademas deben tener todos dimension 1 para que se trate de una base de un espacio vectorial.

In [42]:
for ev in eig_vectors:
   print("la logitud del VP es:"+str(np.linalg.norm(ev)))

la logitud del VP es:1.0
la logitud del VP es:1.0
la logitud del VP es:0.9999999999999999
la logitud del VP es:0.9999999999999998


* para calcular la norma del vector se usa [Frobenius Norm](https://datascience.stackexchange.com/questions/73356/trying-to-understand-the-result-provided-by-np-linalg-norm-function-in-numpy-no)

Para decidir que vectores propios se pueden eliminar sin perder demasiada informacion, para construir un espacio de dimension inferior.:
* Los vectores propios que tengan el valor propio con menos información de la distribucion de los datos son los que se eliminan. Haciendo un ranking descendente de los valores propios.

In [ ]:
eigen_pairs = list(zip(np.abs(eig_vals),eig_vectors))


In [78]:
eigen_pairs = [(np.abs(eig_vals[i]),eig_vectors[:,i]) for i in range(len(eig_vals))]

In [79]:
eigen_pairs

[(2.9380850501999953,
  array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654])),
 (0.9201649041624852,
  array([-0.37741762, -0.92329566, -0.02449161, -0.06694199])),
 (0.14774182104494754,
  array([-0.71956635,  0.24438178,  0.14212637,  0.63427274])),
 (0.020853862176463244,
  array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))]

In [80]:
# se ordenan los vectories propios con los valores propios de
# mayor magnitud
eigen_pairs.sort()

cuantas componentes principales deberia elegir?
* varianza explicativa: nos dice cuanta varianza (informecion) se pude atribuir a cada una de las componentes principales. Que porcentaje de la variabilidad total quedaría explicada si me quedo con 1,2,3 etc.

In [70]:
total_sum = sum(eig_vals_corr)
var_exp = [(i/total_sum)*100 for i in sorted(eig_vals_corr,reverse=True)]
cum_var_exp = np.cumsum(var_exp)

In [63]:
var_exp

[72.96244541329987, 22.85076178670176, 3.668921889282865, 0.5178709107154977]

In [ ]:
cum_var_exp

In [73]:
import plotly.graph_objects as go
plot1 =  px.bar(y=var_exp)
plot2 = px.scatter(y=cum_var_exp)
# Crear la figura final con ambos gráficos
fig = go.Figure(data=plot1.data + plot2.data)

# Actualizar el layout para ajustar los ejes y el título
fig.update_layout(
    xaxis_title="Componentes principales",
    yaxis_title="Porcentaje de varianza explicada",
    title="Porcentaje de Variabilidad explicada para cada componente principal"
)

* Vamos a pasar de un espacio vectorial de dimesion 4 a uno de dimension 2 eligiendo lso dos vectores propios con mayor valor propio (explicacion de la varianza)

In [74]:
eigen_pairs[0][1]

array([ 0.20623499, -0.01977181, -0.75544258,  0.62159691])

In [81]:
W = np.hstack((eigen_pairs[0][1].reshape(4,1),
              eigen_pairs[1][1].reshape(4,1)))

In [82]:
W

array([[ 0.26128628, -0.71956635],
       [-0.12350962,  0.24438178],
       [-0.80144925,  0.14212637],
       [ 0.52359713,  0.63427274]])